In [ ]:
import pandas as pd
from tkinter import Tk, filedialog
import os

# Dicionário de Filiais
filiais = {
    1: "Matriz",
    2: "Juazeiro Norte",
    3: "Sobral",
    5: "São Luís",
    6: "Teresina",
    7: "Maracanaú",
    9: "Parnaíba",
    10: "Juazeiro Bahia",
    11: "Petrolina"
}

# Função para selecionar arquivo
def selecionar_arquivo():
    Tk().withdraw()
    arquivo = filedialog.askopenfilename(filetypes=[("Arquivos Excel", "*.xlsx")])
    return arquivo

# Função para identificar colunas
def identificar_colunas(df):
    cod_produto = ["CODPROD"]
    qunt_sugestao = ["SUG MTZ", "SUG TZ", "SUG MAR"]

    col_cod_produto = next((col for col in df.columns if any(p in col.upper() for p in cod_produto)), None)
    col_qunt_sugestao = next((col for col in df.columns if any(p in col.upper() for p in qunt_sugestao)), None)

    # Se não encontrar, pedir input do usuário
    if not col_cod_produto:
        col_cod_produto = input("Digite o nome da coluna do código do produto: ")
    if not col_qunt_sugestao:
        col_qunt_sugestao = input("Digite o nome da coluna de quantidade do produto: ")

    return col_cod_produto, col_qunt_sugestao

# Função para ler e processar arquivo
def ler_arquivo(arquivo):
    df = pd.read_excel(arquivo)
    print("Colunas detectadas:", df.columns.tolist())

    cod_produto, qunt_sugestao = identificar_colunas(df)

    print(f"Coluna de Código do Produto: {cod_produto}")
    print(f"Coluna de Quantidade do Produto: {qunt_sugestao}")

    return df[[cod_produto, qunt_sugestao]]

# Solicita número da filial
def solicitar_filial():
    while True:
        try:
            num_filial = int(input("Digite o número da filial: "))
            if num_filial in filiais:
                return num_filial, filiais[num_filial]
            else:
                print("Número de filial inválido. Tente novamente.")
        except ValueError:
            print("Entrada inválida. Digite um número válido.")

# Segmentação dos pedidos
def segmentar_pedidos(df, num_filial, nome_filial, caminho_destino):
    df.insert(0, "Filial", num_filial)  # Adiciona a coluna da filial
    num_arquivos = (len(df) // 30) + (1 if len(df) % 30 > 0 else 0)  # Calcula quantos arquivos serão gerados

    for i in range(num_arquivos):
        df_segmento = df.iloc[i * 30:(i + 1) * 30]
        nome_arquivo = f"abastecimento_{nome_filial}_{i+1}.csv"
        caminho_arquivo = os.path.join(caminho_destino, nome_arquivo)
        df_segmento.to_csv(caminho_arquivo, index=False, sep=";")
        print(f"Arquivo salvo: {caminho_arquivo}")

# Seleção do diretório de salvamento
def selecionar_diretorio():
    Tk().withdraw()
    return filedialog.askdirectory()

# Fluxo principal
arquivo_selecionado = selecionar_arquivo()

if arquivo_selecionado:
    df_filtrado = ler_arquivo(arquivo_selecionado)
    num_filial, nome_filial = solicitar_filial()
    caminho_destino = selecionar_diretorio()

    if caminho_destino:
        segmentar_pedidos(df_filtrado, num_filial, nome_filial, caminho_destino)
    else:
        print("Nenhum diretório foi selecionado.")
else:
    print("Nenhum arquivo foi selecionado.")
